Cell 1 — Konfigurasi

In [2]:
from pathlib import Path

# ====== PATHS ======
DATASET_ROOT = Path(r"E:\0.TA_Teguh\GMM Trial 2")
HEAD1_DIR    = DATASET_ROOT / "Head 1"

FPS_ROOT     = Path(r"E:\0.TA_Teguh\Sampling\FPS")
WRS_ROOT     = Path(r"E:\0.TA_Teguh\Sampling\Weighted")

OUT_DIR      = Path(r"E:\0.TA_Teguh\EvalSampling")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# ====== RUN SCOPE ======
SUBJECTS  = list("ABCDEFGHIJ")
N_TARGETS = [48, 56, 64]     # Anda bisa ubah manual

# ====== EVAL SETTINGS ======
USECOLS = ["frame","x","y","z","doppler"]   # evaluasi pakai ini
ROUND_UNIQUE_DECIMALS = 4                  # untuk hitung unique/duplikasi (stabil untuk float)

# Frame filter opsional (biar cepat dulu saat uji)
MAX_FILES_PER_SUBJECT = None   # mis. 2 untuk test cepat; None = semua
MAX_FRAMES_PER_FILE   = None   # mis. 3000 untuk test cepat; None = semua

print("HEAD1_DIR:", HEAD1_DIR)
print("FPS_ROOT :", FPS_ROOT)
print("WRS_ROOT :", WRS_ROOT)
print("OUT_DIR  :", OUT_DIR)
print("N_TARGETS:", N_TARGETS)


HEAD1_DIR: E:\0.TA_Teguh\GMM Trial 2\Head 1
FPS_ROOT : E:\0.TA_Teguh\Sampling\FPS
WRS_ROOT : E:\0.TA_Teguh\Sampling\Weighted
OUT_DIR  : E:\0.TA_Teguh\EvalSampling
N_TARGETS: [48, 56, 64]


Cell 2 — Helper I/O + Validasi file

In [3]:
import pandas as pd

def list_jalan_files(head1_subject_dir: Path):
    files = sorted([p.name for p in head1_subject_dir.glob("Jalan*.csv")])
    return files

def resolve_paths(subject: str, fname: str, n_target: int):
    base_path = HEAD1_DIR / subject / fname
    fps_path  = FPS_ROOT / f"N{n_target}" / subject / fname
    wrs_path  = WRS_ROOT / f"N{n_target}" / subject / fname
    return base_path, fps_path, wrs_path

def read_points_csv(path: Path) -> pd.DataFrame:
    if not path.exists():
        raise FileNotFoundError(str(path))
    df = pd.read_csv(path, usecols=USECOLS)
    # guard types
    df["frame"] = df["frame"].astype(int)
    return df


Cell 3 — Core metrik per frame (5 metrik)

In [4]:
import numpy as np
from sklearn.neighbors import NearestNeighbors

def _to_xyz(df):
    return df[["x","y","z"]].to_numpy(dtype=np.float32)

def _to_xyzd(df):
    return df[["x","y","z","doppler"]].to_numpy(dtype=np.float32)

def centroid_xyz(df):
    X = _to_xyz(df)
    return X.mean(axis=0)

def radial_quantiles(df, mu_xyz, qs=(0.5, 0.9)):
    X = _to_xyz(df)
    r = np.linalg.norm(X - mu_xyz[None, :], axis=1)
    return np.quantile(r, qs)

def nn_p95_baseline_to_sample(df_base, df_samp):
    # Nearest-neighbor distance: for each baseline point, distance to nearest sampled point
    Xb = _to_xyz(df_base)
    Xs = _to_xyz(df_samp)
    if len(Xs) == 0 or len(Xb) == 0:
        return np.nan, np.nan
    nn = NearestNeighbors(n_neighbors=1, algorithm="auto")
    nn.fit(Xs)
    d, _ = nn.kneighbors(Xb, return_distance=True)
    d = d.reshape(-1)
    return float(d.mean()), float(np.quantile(d, 0.95))

def unique_and_maxdup(df_samp, decimals=4):
    """
    Robust unique counting for float point clouds.
    Uses rounding + tuple keys, avoids numpy .view() contiguity issues.
    Unique definition: (x,y,z,doppler) rounded to `decimals`.
    """
    if df_samp is None or len(df_samp) == 0:
        return 0, 0, 0.0

    X = df_samp[["x","y","z","doppler"]].to_numpy(dtype=np.float32, copy=True)
    Xr = np.round(X, decimals=decimals)

    # Build hashable keys per row (safe)
    keys = [tuple(row) for row in Xr]
    # Count duplicates
    from collections import Counter
    c = Counter(keys)

    n_unique = len(c)
    max_dup = max(c.values()) if n_unique > 0 else 0
    unique_ratio = n_unique / len(keys)

    return int(n_unique), int(max_dup), float(unique_ratio)


def frame_metrics(df_base_f, df_samp_f):
    """
    Returns:
      shape_p50_diff, shape_p90_diff,
      centroid_drift,
      nn_mean, nn_p95,
      unique_ratio, max_dup
    """
    if len(df_base_f) == 0:
        return {
            "shape_p50_diff": np.nan, "shape_p90_diff": np.nan,
            "centroid_drift": np.nan,
            "nn_mean": np.nan, "nn_p95": np.nan,
            "unique_ratio": np.nan, "max_dup": np.nan
        }
    if len(df_samp_f) == 0:
        return {
            "shape_p50_diff": np.nan, "shape_p90_diff": np.nan,
            "centroid_drift": np.nan,
            "nn_mean": np.nan, "nn_p95": np.nan,
            "unique_ratio": 0.0, "max_dup": 0
        }

    mu_b = centroid_xyz(df_base_f)
    mu_s = centroid_xyz(df_samp_f)
    centroid_drift = float(np.linalg.norm(mu_s - mu_b))

    # Shape preservation: compare radial quantiles w.r.t baseline centroid
    q_b = radial_quantiles(df_base_f, mu_b, qs=(0.5, 0.9))
    q_s = radial_quantiles(df_samp_f, mu_b, qs=(0.5, 0.9))
    shape_p50_diff = float(abs(q_s[0] - q_b[0]))
    shape_p90_diff = float(abs(q_s[1] - q_b[1]))

    # Coverage: NN baseline -> sample
    nn_mean, nn_p95 = nn_p95_baseline_to_sample(df_base_f, df_samp_f)

    # Dup health (upsampling relevant, but we compute always)
    _, max_dup, unique_ratio = unique_and_maxdup(df_samp_f, decimals=ROUND_UNIQUE_DECIMALS)

    return {
        "shape_p50_diff": shape_p50_diff,
        "shape_p90_diff": shape_p90_diff,
        "centroid_drift": centroid_drift,
        "nn_mean": nn_mean,
        "nn_p95": nn_p95,
        "unique_ratio": unique_ratio,
        "max_dup": max_dup
    }


Cell 4 — Temporal metric (jitter centroid antar frame)

In [5]:
def temporal_jitter(df, frames_sorted):
    # std dari ||mu_t - mu_{t-1}|| (centroid step norm)
    mus = []
    for fr in frames_sorted:
        d = df[df["frame"] == fr]
        if len(d) == 0:
            mus.append(None)
        else:
            mus.append(centroid_xyz(d))

    steps = []
    prev = None
    for mu in mus:
        if mu is None:
            prev = None
            continue
        if prev is not None:
            steps.append(float(np.linalg.norm(mu - prev)))
        prev = mu
    if len(steps) == 0:
        return np.nan
    return float(np.std(np.array(steps, dtype=np.float32)))


Cell 5 — Evaluasi per file (baseline vs FPS vs WRS) + split downsample/upsample

In [6]:
def evaluate_one_file(subject: str, fname: str, n_target: int):
    base_path, fps_path, wrs_path = resolve_paths(subject, fname, n_target)

    df_b = read_points_csv(base_path)
    df_f = read_points_csv(fps_path)
    df_w = read_points_csv(wrs_path)

    # optional limit frames for quick tests
    if MAX_FRAMES_PER_FILE is not None:
        frames_all = sorted(df_b["frame"].unique())[:MAX_FRAMES_PER_FILE]
        df_b = df_b[df_b["frame"].isin(frames_all)]
        df_f = df_f[df_f["frame"].isin(frames_all)]
        df_w = df_w[df_w["frame"].isin(frames_all)]
    frames = sorted(df_b["frame"].unique())

    # per-frame counts baseline (valid frames baseline)
    cnt_b = df_b.groupby("frame").size().to_dict()

    rows = []
    for fr in frames:
        dbf = df_b[df_b["frame"] == fr]
        dff = df_f[df_f["frame"] == fr]
        dwf = df_w[df_w["frame"] == fr]

        nb = int(len(dbf))
        mode = "equal"
        if nb > n_target: mode = "down"
        elif nb < n_target: mode = "up"

        mf = frame_metrics(dbf, dff)
        mw = frame_metrics(dbf, dwf)

        rows.append({
            "subject": subject, "file": fname, "N_target": n_target, "frame": fr, "mode": mode,
            "base_n": nb,
            # FPS metrics
            "fps_shape_p50_diff": mf["shape_p50_diff"],
            "fps_shape_p90_diff": mf["shape_p90_diff"],
            "fps_centroid_drift": mf["centroid_drift"],
            "fps_nn_mean": mf["nn_mean"],
            "fps_nn_p95": mf["nn_p95"],
            "fps_unique_ratio": mf["unique_ratio"],
            "fps_max_dup": mf["max_dup"],
            # WRS metrics
            "wrs_shape_p50_diff": mw["shape_p50_diff"],
            "wrs_shape_p90_diff": mw["shape_p90_diff"],
            "wrs_centroid_drift": mw["centroid_drift"],
            "wrs_nn_mean": mw["nn_mean"],
            "wrs_nn_p95": mw["nn_p95"],
            "wrs_unique_ratio": mw["unique_ratio"],
            "wrs_max_dup": mw["max_dup"],
        })

    df_frame = pd.DataFrame(rows)

    # temporal jitter (per method)
    jitter_base = temporal_jitter(df_b, frames)
    jitter_fps  = temporal_jitter(df_f, frames)
    jitter_wrs  = temporal_jitter(df_w, frames)

    # aggregate summary (overall + per mode)
    def agg_block(d):
        return {
            "frames": int(d["frame"].nunique()),
            "shape_p50_diff_mean": float(d["fps_shape_p50_diff"].mean()),
            "shape_p90_diff_mean": float(d["fps_shape_p90_diff"].mean()),
            "centroid_drift_mean": float(d["fps_centroid_drift"].mean()),
            "nn_p95_mean": float(d["fps_nn_p95"].mean()),
            "unique_ratio_mean": float(d["fps_unique_ratio"].mean()),
            "max_dup_p95": float(np.nanquantile(d["fps_max_dup"], 0.95)),
        }

    def agg_block_w(d):
        return {
            "frames": int(d["frame"].nunique()),
            "shape_p50_diff_mean": float(d["wrs_shape_p50_diff"].mean()),
            "shape_p90_diff_mean": float(d["wrs_shape_p90_diff"].mean()),
            "centroid_drift_mean": float(d["wrs_centroid_drift"].mean()),
            "nn_p95_mean": float(d["wrs_nn_p95"].mean()),
            "unique_ratio_mean": float(d["wrs_unique_ratio"].mean()),
            "max_dup_p95": float(np.nanquantile(d["wrs_max_dup"], 0.95)),
        }

    # overall
    fps_overall = agg_block(df_frame)
    wrs_overall = agg_block_w(df_frame)

    # by mode
    fps_down = agg_block(df_frame[df_frame["mode"]=="down"]) if (df_frame["mode"]=="down").any() else None
    fps_up   = agg_block(df_frame[df_frame["mode"]=="up"])   if (df_frame["mode"]=="up").any() else None
    wrs_down = agg_block_w(df_frame[df_frame["mode"]=="down"]) if (df_frame["mode"]=="down").any() else None
    wrs_up   = agg_block_w(df_frame[df_frame["mode"]=="up"])   if (df_frame["mode"]=="up").any() else None

    summary = {
        "subject": subject, "file": fname, "N_target": n_target,
        "jitter_base": jitter_base, "jitter_fps": jitter_fps, "jitter_wrs": jitter_wrs,
        "fps_overall": fps_overall, "wrs_overall": wrs_overall,
        "fps_down": fps_down, "fps_up": fps_up,
        "wrs_down": wrs_down, "wrs_up": wrs_up,
    }
    return df_frame, summary


Cell 6 — Run evaluasi untuk semua subject + semua file + semua N, simpan hasil

In [7]:
all_summaries = []
all_frames_out = []

for n_target in N_TARGETS:
    for subj in SUBJECTS:
        subj_dir = HEAD1_DIR / subj
        files = list_jalan_files(subj_dir)
        if MAX_FILES_PER_SUBJECT is not None:
            files = files[:MAX_FILES_PER_SUBJECT]

        for fname in files:
            try:
                df_frame, summ = evaluate_one_file(subj, fname, n_target)

                # simpan per-file per-N frame metrics (opsional: besar)
                out_frame_path = OUT_DIR / f"frame_metrics_N{n_target}_{subj}_{fname.replace('.csv','')}.csv"
                df_frame.to_csv(out_frame_path, index=False)

                # siapkan ringkasan tabular untuk agregasi
                def pack(kind, block):
                    if block is None:
                        return None
                    return {
                        "subject": subj, "file": fname, "N_target": n_target, "kind": kind,
                        "frames": block["frames"],
                        "shape_p50_diff_mean": block["shape_p50_diff_mean"],
                        "shape_p90_diff_mean": block["shape_p90_diff_mean"],
                        "centroid_drift_mean": block["centroid_drift_mean"],
                        "nn_p95_mean": block["nn_p95_mean"],
                        "unique_ratio_mean": block["unique_ratio_mean"],
                        "max_dup_p95": block["max_dup_p95"],
                        "jitter_base": summ["jitter_base"],
                        "jitter_fps": summ["jitter_fps"],
                        "jitter_wrs": summ["jitter_wrs"],
                    }

                # FPS blocks
                all_summaries.append(pack("fps_overall", summ["fps_overall"]))
                all_summaries.append(pack("fps_down",    summ["fps_down"]))
                all_summaries.append(pack("fps_up",      summ["fps_up"]))

                # WRS blocks
                bw = summ["wrs_overall"]
                all_summaries.append({
                    "subject": subj, "file": fname, "N_target": n_target, "kind": "wrs_overall",
                    "frames": bw["frames"],
                    "shape_p50_diff_mean": bw["shape_p50_diff_mean"],
                    "shape_p90_diff_mean": bw["shape_p90_diff_mean"],
                    "centroid_drift_mean": bw["centroid_drift_mean"],
                    "nn_p95_mean": bw["nn_p95_mean"],
                    "unique_ratio_mean": bw["unique_ratio_mean"],
                    "max_dup_p95": bw["max_dup_p95"],
                    "jitter_base": summ["jitter_base"],
                    "jitter_fps": summ["jitter_fps"],
                    "jitter_wrs": summ["jitter_wrs"],
                })

                if summ["wrs_down"] is not None:
                    bd = summ["wrs_down"]
                    all_summaries.append({
                        "subject": subj, "file": fname, "N_target": n_target, "kind": "wrs_down",
                        "frames": bd["frames"],
                        "shape_p50_diff_mean": bd["shape_p50_diff_mean"],
                        "shape_p90_diff_mean": bd["shape_p90_diff_mean"],
                        "centroid_drift_mean": bd["centroid_drift_mean"],
                        "nn_p95_mean": bd["nn_p95_mean"],
                        "unique_ratio_mean": bd["unique_ratio_mean"],
                        "max_dup_p95": bd["max_dup_p95"],
                        "jitter_base": summ["jitter_base"],
                        "jitter_fps": summ["jitter_fps"],
                        "jitter_wrs": summ["jitter_wrs"],
                    })

                if summ["wrs_up"] is not None:
                    bu = summ["wrs_up"]
                    all_summaries.append({
                        "subject": subj, "file": fname, "N_target": n_target, "kind": "wrs_up",
                        "frames": bu["frames"],
                        "shape_p50_diff_mean": bu["shape_p50_diff_mean"],
                        "shape_p90_diff_mean": bu["shape_p90_diff_mean"],
                        "centroid_drift_mean": bu["centroid_drift_mean"],
                        "nn_p95_mean": bu["nn_p95_mean"],
                        "unique_ratio_mean": bu["unique_ratio_mean"],
                        "max_dup_p95": bu["max_dup_p95"],
                        "jitter_base": summ["jitter_base"],
                        "jitter_fps": summ["jitter_fps"],
                        "jitter_wrs": summ["jitter_wrs"],
                    })

                print(f"[OK] N{n_target} {subj} {fname}")

            except Exception as e:
                print(f"[SKIP] N{n_target} {subj} {fname} -> {type(e).__name__}: {e}")

df_summary = pd.DataFrame([r for r in all_summaries if r is not None])
out_summary_path = OUT_DIR / "summary_scorecard_all.csv"
df_summary.to_csv(out_summary_path, index=False)

print("Saved:", out_summary_path)
df_summary.head(10)


[OK] N48 A Jalan1.csv
[OK] N48 A Jalan10.csv
[OK] N48 A Jalan11.csv
[OK] N48 A Jalan12.csv
[OK] N48 A Jalan13.csv
[OK] N48 A Jalan14.csv
[OK] N48 A Jalan15.csv
[OK] N48 A Jalan16.csv
[OK] N48 A Jalan17.csv
[OK] N48 A Jalan18.csv
[OK] N48 A Jalan19.csv
[OK] N48 A Jalan2.csv
[OK] N48 A Jalan20.csv
[OK] N48 A Jalan21.csv
[OK] N48 A Jalan22.csv
[OK] N48 A Jalan23.csv
[OK] N48 A Jalan24.csv
[OK] N48 A Jalan25.csv
[OK] N48 A Jalan26.csv
[OK] N48 A Jalan27.csv
[OK] N48 A Jalan28.csv
[OK] N48 A Jalan29.csv
[OK] N48 A Jalan3.csv
[OK] N48 A Jalan30.csv
[OK] N48 A Jalan31.csv
[OK] N48 A Jalan32.csv
[OK] N48 A Jalan33.csv
[OK] N48 A Jalan34.csv
[OK] N48 A Jalan35.csv
[OK] N48 A Jalan36.csv
[OK] N48 A Jalan37.csv
[OK] N48 A Jalan38.csv
[OK] N48 A Jalan39.csv
[OK] N48 A Jalan4.csv
[OK] N48 A Jalan40.csv
[OK] N48 A Jalan41.csv
[OK] N48 A Jalan42.csv
[OK] N48 A Jalan43.csv
[OK] N48 A Jalan44.csv
[OK] N48 A Jalan45.csv
[OK] N48 A Jalan46.csv
[OK] N48 A Jalan47.csv
[OK] N48 A Jalan48.csv
[OK] N48 A Jala

,subject,file,N_target,kind,frames,shape_p50_diff_mean,shape_p90_diff_mean,centroid_drift_mean,nn_p95_mean,unique_ratio_mean,max_dup_p95,jitter_base,jitter_fps,jitter_wrs
0,A,Jalan1.csv,48,fps_overall,82,0.010388,0.042235,0.026423,0.007001,0.617378,12.95,0.615389,0.608574,0.608526
1,A,Jalan1.csv,48,fps_down,17,0.007994,0.010804,0.023426,0.033768,1.000000,1.00,0.615389,0.608574,0.608526
2,A,Jalan1.csv,48,fps_up,63,0.011364,0.052057,0.028070,0.000000,0.501984,13.00,0.615389,0.608574,0.608526
3,A,Jalan1.csv,48,wrs_overall,82,0.009740,0.041647,0.024779,0.009542,0.617378,12.95,0.615389,0.608574,0.608526
4,A,Jalan1.csv,48,wrs_down,17,0.004867,0.007967,0.015495,0.046025,1.000000,1.00,0.615389,0.608574,0.608526
5,A,Jalan1.csv,48,wrs_up,63,0.011364,0.052057,0.028070,0.000000,0.501984,13.00,0.615389,0.608574,0.608526
6,A,Jalan10.csv,48,fps_overall,87,0.009342,0.027652,0.030168,0.010827,0.691331,13.00,0.500190,0.508407,0.503886
7,A,Jalan10.csv,48,fps_down,35,0.013015,0.034640,0.044480,0.026914,1.000000,1.00,0.500190,0.508407,0.503886
8,A,Jalan10.csv,48,fps_up,51,0.007004,0.023398,0.020939,0.000000,0.473448,13.00,0.500190,0.508407,0.503886
9,A,Jalan10.csv,48,wrs_overall,87,0.011861,0.023205,0.020838,0.019672,0.691331,13.00,0.500190,0.508407,0.503886


Cell 7 — Tampilkan perbandingan FPS vs WRS

In [8]:
def summarize_compare(df_summary, n_target, kind_suffix="overall"):
    fps = df_summary[(df_summary["N_target"]==n_target) & (df_summary["kind"]==f"fps_{kind_suffix}")]
    wrs = df_summary[(df_summary["N_target"]==n_target) & (df_summary["kind"]==f"wrs_{kind_suffix}")]

    # gabung per subject+file
    merged = fps.merge(
        wrs,
        on=["subject","file","N_target","frames","jitter_base","jitter_fps","jitter_wrs"],
        suffixes=("_fps","_wrs"),
        how="inner"
    )
    # agregasi global (mean)
    out = {
        "N_target": n_target,
        "kind": kind_suffix,
        "shape_p50_diff_mean_fps": float(merged["shape_p50_diff_mean_fps"].mean()),
        "shape_p50_diff_mean_wrs": float(merged["shape_p50_diff_mean_wrs"].mean()),
        "shape_p90_diff_mean_fps": float(merged["shape_p90_diff_mean_fps"].mean()),
        "shape_p90_diff_mean_wrs": float(merged["shape_p90_diff_mean_wrs"].mean()),
        "centroid_drift_mean_fps": float(merged["centroid_drift_mean_fps"].mean()),
        "centroid_drift_mean_wrs": float(merged["centroid_drift_mean_wrs"].mean()),
        "nn_p95_mean_fps": float(merged["nn_p95_mean_fps"].mean()),
        "nn_p95_mean_wrs": float(merged["nn_p95_mean_wrs"].mean()),
        "unique_ratio_mean_fps": float(merged["unique_ratio_mean_fps"].mean()),
        "unique_ratio_mean_wrs": float(merged["unique_ratio_mean_wrs"].mean()),
        "max_dup_p95_fps": float(merged["max_dup_p95_fps"].mean()),
        "max_dup_p95_wrs": float(merged["max_dup_p95_wrs"].mean()),
        "jitter_ratio_fps": float((merged["jitter_fps"] / (merged["jitter_base"] + 1e-9)).mean()),
        "jitter_ratio_wrs": float((merged["jitter_wrs"] / (merged["jitter_base"] + 1e-9)).mean()),
        "n_files": int(len(merged))
    }
    return out

rows = []
for n in N_TARGETS:
    rows.append(summarize_compare(df_summary, n, "overall"))
    rows.append(summarize_compare(df_summary, n, "down"))
    rows.append(summarize_compare(df_summary, n, "up"))

df_compare = pd.DataFrame(rows)
out_compare_path = OUT_DIR / "compare_fps_vs_wrs.csv"
df_compare.to_csv(out_compare_path, index=False)

print("Saved:", out_compare_path)
df_compare


Saved: E:\0.TA_Teguh\EvalSampling\compare_fps_vs_wrs.csv


,N_target,kind,shape_p50_diff_mean_fps,shape_p50_diff_mean_wrs,shape_p90_diff_mean_fps,shape_p90_diff_mean_wrs,centroid_drift_mean_fps,centroid_drift_mean_wrs,nn_p95_mean_fps,nn_p95_mean_wrs,unique_ratio_mean_fps,unique_ratio_mean_wrs,max_dup_p95_fps,max_dup_p95_wrs,jitter_ratio_fps,jitter_ratio_wrs,n_files
0,48,overall,0.011009,0.010432,0.051058,0.052331,0.034050,0.030560,0.012112,0.021832,0.692589,0.692589,11.409028,11.409028,0.995412,0.997628,720
1,48,down,0.010893,0.009169,0.022765,0.025700,0.031921,0.022577,0.029156,0.050647,1.000000,1.000000,1.000000,1.000000,0.996021,0.998304,699
2,48,up,0.010829,0.010829,0.067040,0.067040,0.034782,0.034782,0.000000,0.000000,0.488042,0.488042,12.762917,12.762917,0.995412,0.997628,720
3,56,overall,0.010511,0.009708,0.049412,0.049739,0.032865,0.028823,0.008328,0.014776,0.646926,0.646926,13.019653,13.019653,0.995250,0.997135,720
4,56,down,0.010721,0.008158,0.021796,0.022686,0.033651,0.020773,0.025564,0.043356,1.000000,1.000000,1.000000,1.000000,0.996607,0.998594,683
5,56,up,0.010040,0.010040,0.059826,0.059826,0.031677,0.031677,0.000000,0.000000,0.486627,0.486627,14.319375,14.319375,0.995250,0.997135,720
6,64,overall,0.010074,0.009210,0.049555,0.049170,0.031481,0.027352,0.005619,0.009731,0.602552,0.602552,14.661667,14.661667,0.994959,0.996422,720
7,64,down,0.010986,0.007686,0.022077,0.020953,0.035118,0.018666,0.022201,0.037017,1.000000,1.000000,1.000000,1.000000,0.996594,0.998173,667
8,64,up,0.009450,0.009450,0.056159,0.056159,0.029455,0.029455,0.000000,0.000000,0.478446,0.478446,15.755417,15.755417,0.994959,0.996422,720
